scraping github.com/topics 
We are getting a list of topics including topic title, topic page url, topic description
for each topic, we are getting top 25 repositories in the topic from the page
for each repository collecting repo name, username, stars, repo url



In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

In [2]:
url = 'https://github.com/topics'

In [3]:
response = requests.get(url)
response.status_code

200

In [4]:
page_cont = response.text


In [5]:
doc = BeautifulSoup(page_cont,'html.parser')


Find topic title tags

In [6]:
topic_title_tags = doc.find_all('p', class_='f3 lh-condensed mb-0 mt-1 Link--primary')


Find topic description tags

In [7]:

topic_desc_tags = doc.find_all('p', class_='f5 color-fg-muted mb-0 mt-1')


In [8]:
topic_title_tags0 = topic_title_tags[0]


Find topic link tags

In [9]:

topic_link_tags = doc.find_all('a', class_='no-underline flex-1 d-flex flex-column')


In [10]:
topic_link_tags[0]['href']

'/topics/3d'

In [11]:
url1 ='https://github.com'
url1 + topic_link_tags[0]['href']

'https://github.com/topics/3d'

Find all the topic in list format

In [12]:

                               
topic_list =[]


for tags in topic_title_tags:
    topic_list.append(tags.text)
    
len(topic_list)

30

In [13]:

                               
topic_desc = []

for tags in topic_desc_tags:
    topic_desc.append(tags.text.strip())
    
len(topic_desc)

30

In [15]:
                               
topic_url =[]

for tags in topic_link_tags:
    topic_url.append(url1 + tags['href'])
    
len(topic_url)


30

In [16]:
topic_dict = {
    'title': topic_list,
    'description': topic_desc,
    'url':topic_url
}

In [17]:
topics_df = pd.DataFrame(topic_dict)

In [18]:
topics_df[:5]

,title,description,url
0,3D,3D modeling is the process of virtually develo...,https://github.com/topics/3d
1,Ajax,Ajax is a technique for creating interactive w...,https://github.com/topics/ajax
2,Algorithm,Algorithms are self-contained sequences that c...,https://github.com/topics/algorithm
3,Amp,Amp is a non-blocking concurrency library for ...,https://github.com/topics/amphp
4,Android,Android is an operating system built by Google...,https://github.com/topics/android


In [19]:
topics_df.to_csv('github_topics.csv',index=None)

In [20]:
topic_page_url = topic_url[0]
topic_page_url

'https://github.com/topics/3d'

In [21]:
response = requests.get(topic_page_url)
response.status_code

200

In [22]:
topic_doc = BeautifulSoup(response.text,'html.parser')

In [23]:
repo_tags = topic_doc.find_all('h3',class_='f3 color-fg-muted text-normal lh-condensed')
len(repo_tags)

30

In [24]:
repo_tags[0]

<h3 class="f3 color-fg-muted text-normal lh-condensed">
<a data-ga-click="Explore, go to repository owner, location:explore feed" data-hydro-click='{"event_type":"explore.click","payload":{"click_context":"REPOSITORY_CARD","click_target":"OWNER","click_visual_representation":"REPOSITORY_OWNER_HEADING","actor_id":null,"record_id":97088,"originating_url":"https://github.com/topics/3d","user_id":null}}' data-hydro-click-hmac="4bdbc49d3c05ae7f70b531fbce709a384200b0768554e0172950286a8db30940" data-turbo="false" data-view-component="true" href="/mrdoob">
  
            mrdoob

  
</a>          /
          <a class="text-bold wb-break-word" data-ga-click="Explore, go to repository, location:explore feed" data-hydro-click='{"event_type":"explore.click","payload":{"click_context":"REPOSITORY_CARD","click_target":"REPOSITORY","click_visual_representation":"REPOSITORY_NAME_HEADING","actor_id":null,"record_id":576201,"originating_url":"https://github.com/topics/3d","user_id":null}}' data-hydro-cli

In [25]:
a_tags = repo_tags[0].find_all('a')
a_tags

[<a data-ga-click="Explore, go to repository owner, location:explore feed" data-hydro-click='{"event_type":"explore.click","payload":{"click_context":"REPOSITORY_CARD","click_target":"OWNER","click_visual_representation":"REPOSITORY_OWNER_HEADING","actor_id":null,"record_id":97088,"originating_url":"https://github.com/topics/3d","user_id":null}}' data-hydro-click-hmac="4bdbc49d3c05ae7f70b531fbce709a384200b0768554e0172950286a8db30940" data-turbo="false" data-view-component="true" href="/mrdoob">
   
             mrdoob
 
   
 </a>,
 <a class="text-bold wb-break-word" data-ga-click="Explore, go to repository, location:explore feed" data-hydro-click='{"event_type":"explore.click","payload":{"click_context":"REPOSITORY_CARD","click_target":"REPOSITORY","click_visual_representation":"REPOSITORY_NAME_HEADING","actor_id":null,"record_id":576201,"originating_url":"https://github.com/topics/3d","user_id":null}}' data-hydro-click-hmac="517d3d5cb9d89752156923904a4238816bc9b51ab7772f3e3644ce897d8d

In [26]:
a_tags[0].text.strip()

'mrdoob'

In [27]:
a_tags[1].text.strip()

'three.js'

In [28]:

repo_url = url1+ a_tags[0]['href']
repo_url

'https://github.com/mrdoob'

In [29]:
star_tags = topic_doc.find_all('span',class_='Counter js-social-count')
len(star_tags)

30

In [30]:
star_tags[0].text.strip()

'80.9k'

Creating a function to define the number of views in numbers

In [31]:
def parse_star_count(stars_str):
    stars_str = stars_str.strip()
    if stars_str[-1]=='k':
        return int(float(stars_str[:-1])*1000)
    return int(stars_str)

In [32]:
parse_star_count(star_tags[0].text.strip())

80900

In [33]:
#REturn all the required info about a repository

def get_repo_info(h3_tag, star_tag):
    a_tags = h3_tag.find_all('a')
    username = a_tags[0].text.strip()
    repo_name =a_tags[1].text.strip()
    repo_url = url1 + a_tags[1]['href']
    stars = parse_star_count(star_tag.text.strip())
    return username, repo_name, stars, repo_url

In [34]:
get_repo_info(repo_tags[0],star_tags[0])

('mrdoob', 'three.js', 80900, 'https://github.com/mrdoob/three.js')

In [35]:
topic_repo_dict ={
    'username':[],
    'repo_name':[],
    'stars':[],
    'repo_url': []
}
for i in range(len(repo_tags)):
    repo_info =get_repo_info(repo_tags[i],star_tags[i])
    topic_repo_dict['username'].append(repo_info[0])
    topic_repo_dict['repo_name'].append(repo_info[1])
    topic_repo_dict['stars'].append(repo_info[2])
    topic_repo_dict['repo_url'].append(repo_info[3])

In [36]:
#topic_repo_dict

In [38]:
topic_repo_df = pd.DataFrame(topic_repo_dict)
topic_repo_df

,username,repo_name,stars,repo_url
0,mrdoob,three.js,80900,https://github.com/mrdoob/three.js
1,libgdx,libgdx,19800,https://github.com/libgdx/libgdx
2,pmndrs,react-three-fiber,17500,https://github.com/pmndrs/react-three-fiber
3,BabylonJS,Babylon.js,16500,https://github.com/BabylonJS/Babylon.js
4,aframevr,aframe,14000,https://github.com/aframevr/aframe
5,ssloy,tinyrenderer,13500,https://github.com/ssloy/tinyrenderer
6,lettier,3d-game-shaders-for-beginners,12600,https://github.com/lettier/3d-game-shaders-for...
7,FreeCAD,FreeCAD,11100,https://github.com/FreeCAD/FreeCAD
8,metafizzy,zdog,9100,https://github.com/metafizzy/zdog
9,CesiumGS,cesium,8500,https://github.com/CesiumGS/cesium


In [46]:
def get_topic_page(topic_url):
    #Retrieving the page
    response = requests.get(topic_url)
    if response.status_code !=200:
        raise Exception('Failed to load page {}'.format(topic_url))
    # parse using bs4    
    topic_doc = BeautifulSoup(response.text,'html.parser')
    
    return topic_doc
    
def get_repo_info(h3_tag, star_tag):
    # getting all the a tags inside the h3 tag
    a_tags = h3_tag.find_all('a')
    # striping the username info
    username = a_tags[0].text.strip()
    # striping the repo name info
    repo_name =a_tags[1].text.strip()
    #getting the url from href attr of the a tag
    repo_url = url1 + a_tags[1]['href']
    #getting the stars
    stars = parse_star_count(star_tag.text.strip())
    
    return username, repo_name, stars, repo_url


def get_topic_repos(topic_doc):
    # collecting the parent tag of the required tags title, username, url
    repo_tags = topic_doc.find_all('h3',class_='f3 color-fg-muted text-normal lh-condensed')
    # collecting the star tags
    star_tags = topic_doc.find_all('span',class_='Counter js-social-count')
    # creating a dictionary to store all the info
    topic_repo_dict ={
    'username':[],
    'repo_name':[],
    'stars':[],
    'repo_url': []
    }
    # get repo info
    
    for i in range(len(repo_tags)):
        # getting repository info from get_repo_info function
        repo_info = get_repo_info(repo_tags[i],star_tags[i])
        #appending info into the dictionary
        topic_repo_dict['username'].append(repo_info[0])
        topic_repo_dict['repo_name'].append(repo_info[1])
        topic_repo_dict['stars'].append(repo_info[2])
        topic_repo_dict['repo_url'].append(repo_info[3])
        
    return pd.DataFrame(topic_repo_dict)

 # defining a function to store all the repository infos of individual topics in individual CSV files
    
def scrape_topic(topic_url,topic_name):
    fname = topic_name + '.csv'
    # checking if any file name already exists
    if os.path.exists(fname):
        print('The file {} already exists. Skipping...'.format(fname))
        return
    topic_df = get_topic_repos(get_topic_page(topic_url))
    topic_df.to_csv(fname,index=None)
    

In [47]:
get_topic_repos(get_topic_page(topic_url[6]))

,username,repo_name,stars,repo_url
0,ansible,ansible,52800,https://github.com/ansible/ansible
1,trailofbits,algo,25300,https://github.com/trailofbits/algo
2,StreisandEffect,streisand,22800,https://github.com/StreisandEffect/streisand
3,bregman-arie,devops-exercises,22100,https://github.com/bregman-arie/devops-exercises
4,kubernetes-sigs,kubespray,12100,https://github.com/kubernetes-sigs/kubespray
5,ansible,awx,10900,https://github.com/ansible/awx
6,easzlab,kubeasz,8000,https://github.com/easzlab/kubeasz
7,MichaelCade,90DaysOfDevOps,7600,https://github.com/MichaelCade/90DaysOfDevOps
8,geerlingguy,ansible-for-devops,5600,https://github.com/geerlingguy/ansible-for-devops
9,khuedoan,homelab,5600,https://github.com/khuedoan/homelab


Write one function to the following:
1. Get the lst of all the topics from topics page
2. get the list of top repos from individual topics
3. for each topic create a CSV of the top repos for the topic

In [41]:
def get_topic_titles(doc):
    topic_title_tags = doc.find_all('p', class_='f3 lh-condensed mb-0 mt-1 Link--primary')
    # getting all the topic titles
    topic_list =[]
    for tags in topic_title_tags:
        topic_list.append(tags.text)
        
    return topic_list

def get_topic_desc(doc):
    topic_desc_tags = doc.find_all('p', class_='f5 color-fg-muted mb-0 mt-1')
    #getting topic descriptions
    topic_desc = []
    for tags in topic_desc_tags:
        topic_desc.append(tags.text.strip())
        
    return topic_desc

def get_topic_url(doc):
    topic_link_tags = doc.find_all('a', class_='no-underline flex-1 d-flex flex-column')
    #getting topic urls
    topic_url =[]

    for tags in topic_link_tags:
        topic_url.append(url1 + tags['href'])
    return topic_url

    

def scrape_topics():
    topic_url = 'https://github.com/topics'
    requests.get(topic_url)
    if response.status_code !=200:
        raise Exception('Failed to load page {}'.format(topic_url))
        
    topics_dict= {
        'title': get_topic_titles(doc),
        'description': get_topic_desc(doc),
        'url': get_topic_url(doc)
    }
        
    return pd.DataFrame(topics_dict)
    

In [42]:
for index, row in topics_df.iterrows():
        print(row['title'],row['url'])

3D https://github.com/topics/3d
Ajax https://github.com/topics/ajax
Algorithm https://github.com/topics/algorithm
Amp https://github.com/topics/amphp
Android https://github.com/topics/android
Angular https://github.com/topics/angular
Ansible https://github.com/topics/ansible
API https://github.com/topics/api
Arduino https://github.com/topics/arduino
ASP.NET https://github.com/topics/aspnet
Atom https://github.com/topics/atom
Awesome Lists https://github.com/topics/awesome
Amazon Web Services https://github.com/topics/aws
Azure https://github.com/topics/azure
Babel https://github.com/topics/babel
Bash https://github.com/topics/bash
Bitcoin https://github.com/topics/bitcoin
Bootstrap https://github.com/topics/bootstrap
Bot https://github.com/topics/bot
C https://github.com/topics/c
Chrome https://github.com/topics/chrome
Chrome extension https://github.com/topics/chrome-extension
Command line interface https://github.com/topics/cli
Clojure https://github.com/topics/clojure
Code quality h

In [48]:
def scrape_topic_repos():
    print('Scraping list of topics')
    # collecting all the details of topics
    topics_df = scrape_topics()
    
    # getting all the repo csvs
    for index, row in topics_df.iterrows():
        print('Scraping top repositories for "{}"'.format(row['title']))
        scrape_topic(row['url'],row['title'])
    

In [49]:
scrape_topic_repos()

Scraping list of topics
Scraping top repositories for "3D"
The file 3D.csv already exists. Skipping...
Scraping top repositories for "Ajax"
The file Ajax.csv already exists. Skipping...
Scraping top repositories for "Algorithm"
The file Algorithm.csv already exists. Skipping...
Scraping top repositories for "Amp"
The file Amp.csv already exists. Skipping...
Scraping top repositories for "Android"
The file Android.csv already exists. Skipping...
Scraping top repositories for "Angular"
The file Angular.csv already exists. Skipping...
Scraping top repositories for "Ansible"
The file Ansible.csv already exists. Skipping...
Scraping top repositories for "API"
The file API.csv already exists. Skipping...
Scraping top repositories for "Arduino"
The file Arduino.csv already exists. Skipping...
Scraping top repositories for "ASP.NET"
The file ASP.NET.csv already exists. Skipping...
Scraping top repositories for "Atom"
The file Atom.csv already exists. Skipping...
Scraping top repositories for "